In [1]:
import csv
import pandas as pd
from geopy.distance import geodesic

# 读取地铁站数据
tube_stations = []
with open('tube.csv', 'r') as tube_file:
    tube_reader = csv.DictReader(tube_file)
    for row in tube_reader:
        station_lat = float(row['y'])  # 假设经纬度在'y'列
        station_lng = float(row['x'])  # 假设经纬度在'x'列
        station_name = row['NAME']  # 假设地铁站名在'name'列
        tube_stations.append({'name': station_name, 'lat': station_lat, 'lng': station_lng})

# 读取厕所数据
public_toilets = []
with open('toilets_baby.csv', 'r') as toilet_file:
    toilet_reader = csv.DictReader(toilet_file)
    for row in toilet_reader:
        toilet_lat = float(row['lat'])  # 假设经纬度在'lat'列
        toilet_lng = float(row['lng'])  # 假设经纬度在'lng'列
        public_toilets.append({'lat': toilet_lat, 'lng': toilet_lng})

k = 1  # 系数
beta = 1  # 距离的指数
Mj = 1  # 厕所的服务能力，设为相等

# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility = pd.DataFrame(columns=['Station', 'Accessibility'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility = k * (Mj / distance**beta)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility += accessibility
    
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)

    # 设置 pandas 显示的最大行数和最大列数
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 打印地铁站的可达性表格
print(tube_accessibility)

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('baby_tube1.csv', index=False)



<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/tmp/ipykernel_81239/3590531298.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)
/tmp/ipykernel_81239/3590531298.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)
/tmp/ipykernel_81239/3590531298.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_

                     Station  Accessibility
0                    Aldgate     103.664892
1               Aldgate East      98.527282
2                      Angel      82.563766
3               Baker Street      97.797258
4                       Bank     125.761210
5                   Barbican     103.590641
6                  Bayswater      77.808267
7                Blackfriars     113.034213
8                Bond Street      94.037821
9                    Borough      85.635549
10             Cannon Street     127.581610
11             Chancery Lane      98.728033
12             Charing Cross     125.104646
13             Covent Garden     127.892993
14              Earl's Court      69.145458
15   Edgware Road (Bakerloo)      83.409707
16                Embankment     121.260914
17                    Euston     107.151460
18             Euston Square     110.431061
19                Farringdon      96.404357
20           Gloucester Road      73.199276
21             Goodge Street    

/tmp/ipykernel_81239/3590531298.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility = tube_accessibility.append({'Station': station_name, 'Accessibility': station_accessibility}, ignore_index=True)


In [2]:
# 按可达性降序排序
tube_accessibility = tube_accessibility.sort_values(by='Accessibility', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility)


                     Station  Accessibility
43         Piccadilly Circus     132.019090
31          Leicester Square     128.514883
13             Covent Garden     127.892993
10             Cannon Street     127.581610
4                       Bank     125.761210
12             Charing Cross     125.104646
32          Liverpool Street     123.373788
53                St. Paul's     123.047361
16                Embankment     121.260914
34             Mansion House     119.408918
7                Blackfriars     113.034213
60                  Waterloo     112.968301
61               Westminster     112.839630
59             Warren Street     111.988170
18             Euston Square     110.431061
37                  Monument     110.212964
55      Tottenham Court Road     108.346200
17                    Euston     107.151460
41             Oxford Circus     106.712966
38                  Moorgate     106.183526
54                    Temple     105.631833
27  King's Cross St. Pancras    

In [3]:
# 计算最大值、最小值、平均值和标准差
max_accessibility = tube_accessibility['Accessibility'].max()
min_accessibility = tube_accessibility['Accessibility'].min()
mean_accessibility = tube_accessibility['Accessibility'].mean()
std_accessibility = tube_accessibility['Accessibility'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility}")
print(f"最小可达性值: {min_accessibility}")
print(f"平均可达性值: {mean_accessibility}")
print(f"标准差: {std_accessibility}")

最大可达性值: 132.01908957323315
最小可达性值: 69.14545789218727
平均可达性值: 99.22593383377645
标准差: 15.861851495448686


In [4]:
beta15 = 1.5  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility15 = pd.DataFrame(columns=['Station', 'Accessibility1.5'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility15 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility15 = k * (Mj / distance**beta15)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility15 += accessibility15
        # 在原有的tube.csv文件中新增一个Accessibility列，并将可达性值写入对应行

            
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
# 按可达性降序排序
tube_accessibility15 = tube_accessibility15.sort_values(by='Accessibility1.5', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility15)
# 计算最大值、最小值、平均值和标准差
max_accessibility15 = tube_accessibility15['Accessibility1.5'].max()
min_accessibility15 = tube_accessibility15['Accessibility1.5'].min()
mean_accessibility15 = tube_accessibility15['Accessibility1.5'].mean()
std_accessibility15 = tube_accessibility15['Accessibility1.5'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility15}")
print(f"最小可达性值: {min_accessibility15}")
print(f"平均可达性值: {mean_accessibility15}")
print(f"标准差: {std_accessibility15}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility15, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('baby_tube15.csv', index=False)


/tmp/ipykernel_81239/1478650784.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_81239/1478650784.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_81239/1478650784.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)
/tmp/ipykernel_81239/1478650784.py:27: FutureWarning

                     Station  Accessibility1.5
10             Cannon Street        153.465141
13             Covent Garden        147.549439
43         Piccadilly Circus        146.858675
32          Liverpool Street        141.583013
4                       Bank        141.123775
53                St. Paul's        130.368180
31          Leicester Square        127.564903
12             Charing Cross        119.713422
59             Warren Street        119.182687
60                  Waterloo        118.390017
16                Embankment        117.573305
34             Mansion House        115.529966
61               Westminster        109.719966
18             Euston Square        109.308929
7                Blackfriars         99.278086
37                  Monument         98.833685
41             Oxford Circus         97.250717
17                    Euston         95.950080
0                    Aldgate         89.105236
3               Baker Street         86.441071
27  King's Cr

/tmp/ipykernel_81239/1478650784.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility15 = tube_accessibility15.append({'Station': station_name, 'Accessibility1.5': station_accessibility15}, ignore_index=True)


In [5]:
beta18 = 1.8  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility18 = pd.DataFrame(columns=['Station', 'Accessibility1.8'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility18 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility18 = k * (Mj / distance**beta18)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility18 += accessibility18
    

            
    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
# 按可达性降序排序
tube_accessibility18 = tube_accessibility18.sort_values(by='Accessibility1.8', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility18)
# 计算最大值、最小值、平均值和标准差
max_accessibility18 = tube_accessibility18['Accessibility1.8'].max()
min_accessibility18 = tube_accessibility18['Accessibility1.8'].min()
mean_accessibility18 = tube_accessibility18['Accessibility1.8'].mean()
std_accessibility18 = tube_accessibility18['Accessibility1.8'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility18}")
print(f"最小可达性值: {min_accessibility18}")
print(f"平均可达性值: {mean_accessibility18}")
print(f"标准差: {std_accessibility18}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility18, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('baby_tube18.csv', index=False)

/tmp/ipykernel_81239/1360680000.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_81239/1360680000.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_81239/1360680000.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)
/tmp/ipykernel_81239/1360680000.py:27: FutureWarning

                     Station  Accessibility1.8
10             Cannon Street        227.537829
13             Covent Garden        219.033291
32          Liverpool Street        201.140015
43         Piccadilly Circus        196.751065
4                       Bank        195.903416
53                St. Paul's        176.988899
60                  Waterloo        176.651909
59             Warren Street        173.109320
31          Leicester Square        153.555414
18             Euston Square        148.980470
16                Embankment        148.456591
61               Westminster        145.672722
34             Mansion House        141.477017
12             Charing Cross        141.094619
41             Oxford Circus        130.012947
37                  Monument        118.877895
17                    Euston        118.025557
7                Blackfriars        117.224125
3               Baker Street        113.499463
0                    Aldgate        105.290680
1            

/tmp/ipykernel_81239/1360680000.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility18 = tube_accessibility18.append({'Station': station_name, 'Accessibility1.8': station_accessibility18}, ignore_index=True)


In [6]:
beta2 = 2  # 距离的指数


# 创建一个空的 DataFrame 来存储地铁站的可达性
tube_accessibility2 = pd.DataFrame(columns=['Station', 'Accessibility2'])

# 遍历每个地铁站
for station in tube_stations:
    station_name = station['name']
    station_lat = station['lat']
    station_lng = station['lng']
    station_accessibility2 = 0
    
    # 遍历每个厕所，计算可达性并累加
    for toilet in public_toilets:
        toilet_lat = toilet['lat']
        toilet_lng = toilet['lng']
        distance = geodesic((station_lat, station_lng), (toilet_lat, toilet_lng)).kilometers
        accessibility2 = k * (Mj / distance**beta2)
        
        # 将可达性累加到地铁站的可达性中
        station_accessibility2 += accessibility2
    

    # 将地铁站的可达性添加到 DataFrame 中
    tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
# 按可达性降序排序
tube_accessibility2 = tube_accessibility2.sort_values(by='Accessibility2', ascending=False)

# 打印地铁站的可达性表格
print(tube_accessibility2)
# 计算最大值、最小值、平均值和标准差
max_accessibility2 = tube_accessibility2['Accessibility2'].max()
min_accessibility2 = tube_accessibility2['Accessibility2'].min()
mean_accessibility2 = tube_accessibility2['Accessibility2'].mean()
std_accessibility2 = tube_accessibility2['Accessibility2'].std()

# 打印计算结果
print(f"最大可达性值: {max_accessibility2}")
print(f"最小可达性值: {min_accessibility2}")
print(f"平均可达性值: {mean_accessibility2}")
print(f"标准差: {std_accessibility2}")

# 读取已有的 tube.csv 文件
existing_tube_data = pd.read_csv('tube.csv')

# 合并数据
merged_data = pd.merge(existing_tube_data, tube_accessibility2, left_on='NAME', right_on='Station', how='left')

# 保存合并后的数据到新的 tube.csv 文件
merged_data.to_csv('baby_tube2.csv', index=False)

/tmp/ipykernel_81239/1498133821.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_81239/1498133821.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_81239/1498133821.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
/tmp/ipykernel_81239/1498133821.py:26: FutureWarning: The frame.app

                     Station  Accessibility2
10             Cannon Street      318.560975
13             Covent Garden      311.756433
32          Liverpool Street      272.912488
4                       Bank      261.770014
60                  Waterloo      258.561969
43         Piccadilly Circus      253.806699
59             Warren Street      243.656348
53                St. Paul's      235.847961
18             Euston Square      201.762710
61               Westminster      193.455351
16                Embankment      188.654738
31          Leicester Square      182.788174
41             Oxford Circus      178.193197
34             Mansion House      172.722311
12             Charing Cross      166.059445
3               Baker Street      152.477707
17                    Euston      148.319840
37                  Monument      146.301159
7                Blackfriars      142.768675
0                    Aldgate      128.557230
58                  Victoria      124.769771
1         

/tmp/ipykernel_81239/1498133821.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tube_accessibility2 = tube_accessibility2.append({'Station': station_name, 'Accessibility2': station_accessibility2}, ignore_index=True)
